In [4]:
import os
import re
import pandas as pd
import sqlite3
import numpy as np

# =========================================================
# Database setup
# =========================================================
DB_NAME = "tc_data.db"
conn = sqlite3.connect(DB_NAME)

# =========================================================
# 1. Parse files in folder TC_biendong_625x500-2015-2023_Himawari
# =========================================================
folder_path = "TC_biendong_625x500-2015-2023_Himawari"
folder_path = r"data_from_tien\satellite_2015-2023\satellite\TC_biendong_625x500-2015-2023_Himawari"
image_records = []

# Example filename: 201589N.MELOR.2015.12.11.0600.HIMARAWI-8-9.ir_resize
pattern = re.compile(
    r"(?P<pic_id>\d{7}[A-Z])\.(?P<storm_name>[A-Z]+)\.(?P<year>\d{4})\.(?P<month>\d{2})\.(?P<day>\d{2})\.(?P<hour>\d{4})\.(?P<folder_id>.+)\.ir_resize"
)

for fname in os.listdir(folder_path):
    match = pattern.match(fname)
    if match:
        record = match.groupdict()
        record["file_name"] = fname
        image_records.append(record)

df_images = pd.DataFrame(image_records)

# Save as SQLite table
df_images.to_sql("himawari_images", conn, if_exists="replace", index=False)

# =========================================================
# 2. Load RSMC Best Track CSV
# =========================================================
best_track_file = r"data_from_tien\satellite_2015-2023\satellite\RSMC_Best_Track_Data_2024.csv"
# Ensure empty cells are NaN
df_best_track = pd.read_csv(best_track_file, na_values=[""])

# Save as SQLite table
df_best_track.to_sql("best_track", conn, if_exists="replace", index=False)

# =========================================================
# 3. Load tc_labels.csv and convert bounding boxes to lat/lon
# =========================================================
label_file = r"data_from_tien\satellite_2015-2023\satellite\tc_labels.csv"
df_labels = pd.read_csv(label_file, header=None)
df_labels.columns = ["image_name", "x1", "y1", "x2", "y2", "category"]

# Image dimensions and domain for conversion
img_width = 625
img_height = 500
biendong_startlat = -4.99
biendong_endlat = 34.99
biendong_startlon = 90.01
biendong_endlon = 134.49

def pixels_to_latlon(x, y):
    """Convert pixel coordinates to latitude/longitude."""
    lon = biendong_startlon + (x / img_width) * (biendong_endlon - biendong_startlon)
    lat = biendong_startlat + ((img_height - y) / img_height) * (biendong_endlat - biendong_startlat)
    return lat, lon

# Apply conversion for top-left and bottom-right
df_labels[["lat1", "lon1"]] = df_labels.apply(lambda row: pixels_to_latlon(row["x1"], row["y1"]), axis=1, result_type="expand")
df_labels[["lat2", "lon2"]] = df_labels.apply(lambda row: pixels_to_latlon(row["x2"], row["y2"]), axis=1, result_type="expand")

# Save as SQLite table
df_labels.to_sql("tc_labels", conn, if_exists="replace", index=False)

# =========================================================
# Done
# =========================================================
conn.close()
print("All tables successfully saved to SQLite database:", DB_NAME)


All tables successfully saved to SQLite database: tc_data.db
